In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [2]:
print("Running on: " + str(device))

Running on: mps


In [ ]:
lennet = nn.Sequential(
    # Block 1: 1@28x28 -> 6@28x28 -> 6@14x14
    nn.Conv2d(in_channels=1, out_channels=160, kernel_size=3, padding=1), # 28 - 5 + 2*2 + 1 = 28
    nn.Conv2d(in_channels=160, out_channels=160, kernel_size=3, padding=1), # 28 - 5 + 2*2 + 1 = 28
    nn.Conv2d(in_channels=160, out_channels=160, kernel_size=3, padding=1), # 28 - 5 + 2*2 + 1 = 28
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    
    # Block 2: 6@14x14 -> 16@10x10 -> 16@5x5
    nn.Conv2d(in_channels=160, out_channels=160, kernel_size=5), # 14 - 5 + 2*0 + 1 = 10
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    # Block 3: 16@5x5 -> 130@1x1
    nn.Conv2d(in_channels=160, out_channels=200, kernel_size=5), # 5 - 5kernal + 2*0padding + 1 = 1 
    nn.ReLU(),
    nn.Flatten(),
    
    # Fully connected layers
    nn.Linear(in_features=200, out_features=84),
    nn.ReLU(),
    nn.Linear(in_features=84, out_features=10)
).to(device)

print(lennet)


Sequential(
  (0): Conv2d(1, 160, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(160, 160, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(160, 160, kernel_size=(5, 5), stride=(1, 1))
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(160, 200, kernel_size=(5, 5), stride=(1, 1))
  (9): ReLU()
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=200, out_features=84, bias=True)
  (12): ReLU()
  (13): Linear(in_features=84, out_features=10, bias=True)
)


In [35]:
image = torch.zeros((1, 1, 28, 28), dtype=torch.float32)

In [36]:
lennet.forward(image.to(device))

RuntimeError: Calculated padded input size per channel: (4 x 4). Kernel size: (5 x 5). Kernel size can't be greater than actual input size

In [38]:
nn.ReLU().to(device=device).forward(image.to(device))

tensor([[[[1.1000, 0.0000, 3.0000, 2.0000, 3.0000, 2.0000],
          [1.0000, 2.0000, 3.0000, 2.0000, 3.0000, 2.0000],
          [1.0000, 2.0000, 3.0000, 2.0000, 3.0000, 2.0000],
          [1.0000, 2.0000, 3.0000, 2.0000, 3.0000, 2.0000],
          [1.0000, 2.0000, 3.0000, 2.0000, 3.0000, 2.0000],
          [1.0000, 2.0000, 3.0000, 2.0000, 3.0000, 2.0000]]]], device='mps:0')

In [39]:
print(lennet[0].weight)

Parameter containing:
tensor([[[[ 0.1858, -0.1215, -0.0571],
          [-0.0275,  0.2814,  0.3199],
          [ 0.2991, -0.1204, -0.0814]]]], device='mps:0', requires_grad=True)


In [23]:
print(nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1).weight)

Parameter containing:
tensor([[[[-0.1457,  0.1393,  0.2887],
          [-0.2557,  0.3255, -0.2952],
          [-0.1939,  0.2443, -0.1839]]]], requires_grad=True)
